In [ ]:
import os
import zipfile as zip
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from fuzzywuzzy import fuzz
import ast

In [ ]:
os.chdir("E:\data")

In [ ]:
scpatents = pd.read_csv("scpatents.csv", usecols=['patent_id'])

In [ ]:
scpatents

In [ ]:
scpatents = scpatents.drop_duplicates('patent_id')

In [ ]:
pi = pd.read_csv("patent_inventor.tsv", delimiter="\t")

In [ ]:
pi

In [ ]:
portfolios = pi.groupby(['inventor_id'])

In [ ]:
sci = scpatents.merge(pi, how='left', on='patent_id')

In [ ]:
sci

In [ ]:
sci.to_csv("scinventors.csv")

In [ ]:
sci = pd.read_csv("scinventors.csv", index_col=0)

In [ ]:
sci

In [ ]:
scui = pd.DataFrame(sci['inventor_id'].unique(), columns=['inventor_id'])

In [ ]:
pi[['patent_id','inventor_id']]

In [ ]:
scui = scui.merge(pi[['patent_id','inventor_id']], how='left', on='inventor_id')

In [ ]:
scui.to_csv("scuniqueinventors.csv")

In [ ]:
scui = pd.read_csv("scuniqueinventors.csv", index_col=0)

In [ ]:
ipc = pd.read_csv("ipc3.csv", usecols=["patent_id","class","main_group","subgroup"])

In [ ]:
scui = scui.merge(ipc, how='left', on='patent_id')

In [ ]:
scui = scui[scui['main_group'].notna()]

In [ ]:
scui['main_group'] = scui['main_group'].astype(int)
scui['subgroup'] = scui['subgroup'].astype(int)

In [ ]:
scui

In [ ]:
scui.drop_duplicates(subset=['inventor_id','class','main_group'])

In [ ]:
scui.drop_duplicates(subset=['inventor_id','class'])

In [ ]:
scip = scui.groupby('inventor_id')['class'].unique().apply(list)

In [ ]:
scip = scip.to_frame()

In [ ]:
scip

In [ ]:
scip.to_csv('scip.csv')

In [ ]:
scip = pd.read_csv('scip.csv')

In [ ]:
scip['class'] = scip['class'].apply(ast.literal_eval)

In [ ]:
scip

In [ ]:
scin = scui.groupby('inventor_id')['patent_id'].nunique().to_frame()

In [ ]:
scic = scui.groupby('inventor_id')['class'].nunique().to_frame()

In [ ]:
scic.to_csv("scic.csv")

In [ ]:
scic = pd.read_csv("scic.csv")

In [ ]:
scin.to_csv("scin.csv")

In [ ]:
scin = pd.read_csv("scin.csv")

In [ ]:
scic

In [ ]:
scip = pd.merge(scip,scic)

In [ ]:
scip = scip.rename(columns = {'class':'#class'})

In [ ]:
scip.to_csv('scinventorportfolio.csv')

In [ ]:
scip = pd.read_csv("scinventorportfolio.csv", index_col=0)

In [ ]:
scip

In [ ]:
scip['classes'] = scip['classes'].apply(ast.literal_eval)

In [ ]:
plt.hist(scip['#patents'], range=(0,40))

In [ ]:
plt.scatter(scip['#patents'], scip['#class'])

In [ ]:
scip[scip['#class']>200]

In [ ]:
inventor = pd.read_csv("inventor.tsv", delimiter='\t')

In [ ]:
inventor

In [ ]:
pid = pd.read_csv("persistent_inventor_disambig.tsv", delimiter = '\t', nrows=10000)

In [ ]:
pid

In [ ]:
test = pd.melt(pid, id_vars=['rawinventor_id','disamb_inventor_id_20201229']).drop('variable', axis=1).dropna().groupby('rawinventor_id')['value'].unique()

In [ ]:
test

In [ ]:
test[test.str.len()!=1]

In [ ]:
reader = pd.read_csv("persistent_inventor_disambig.tsv", delimiter = '\t', chunksize=10000)
s = pd.Series()
for chunk in reader:
    result = pd.melt(chunk, id_vars=['rawinventor_id','disamb_inventor_id_20201229']).drop('variable', axis=1).dropna().groupby('rawinventor_id')['value'].unique()
    result = result[result.str.len()!=1]
    s = s.append(result)

In [ ]:
s.to_csv("scdisam.csv")

In [ ]:
s = pd.read_csv("scdisam.csv")

In [ ]:
s = s.rename(columns={'Unnamed: 0':'rawinventor_id','0':'patents'})

In [ ]:
s

In [ ]:
s['patents'] = s['patents'].str.replace(' ',',')

In [ ]:
s

In [ ]:
s = s.drop_duplicates(subset=['patents'])

In [ ]:
s

In [ ]:
s['patents'] = s['patents'].apply(ast.literal_eval)

In [ ]:
s2 = s.explode(column='patents')

In [ ]:
s2

In [ ]:
scip[scip['inventor_id'].isin(s2['patents'])]

In [ ]:
s2[s2['patents']=='10000513-4']

In [ ]:
s2[s2['rawinventor_id']=='0kjv6l9zm8zibc9wzsnrwf76y']

In [ ]:
s2[s2['rawinventor_id']=='79giv1ch72x92eaqmbybiov4y']

In [ ]:
scip[scip['inventor_id']=='9944761-6']

In [ ]:
scip[scip['inventor_id']=='10000513-4']

In [ ]:
scip[scip['inventor_id'].isin(['3969153-1', '5491497-1', '5043297-1'])]

In [ ]:
j = 0
for i in scip['inventor_id'].values:
    print(j)
    if i in s2.values:
        print(i)
        
    j+=1
    if j==10:
        break


In [ ]:
'10000322-2' in scip['inventor_id'].values

In [ ]:
scip['inventor_id']